In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from decimal import Decimal

scale_factor = Decimal("1")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w  = Decimal("0.4")
        top_w  = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def plot_packing(trees, n, title=None):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Plot trees
    for tree in trees:
        x, y = tree.polygon.exterior.xy
        ax.fill(x, y, alpha=0.5, fc='green', ec='black')
        
    # Calculate and plot bounding box
    all_polygons = [t.polygon for t in trees]
    union_poly = unary_union(all_polygons)
    minx, miny, maxx, maxy = union_poly.bounds
    width = maxx - minx
    height = maxy - miny
    side = max(width, height)
    
    rect = plt.Rectangle((minx, miny), width, height, linewidth=2, edgecolor='r', facecolor='none', label='Bounding Box')
    ax.add_patch(rect)
    
    # Square box
    square = plt.Rectangle((minx, miny), side, side, linewidth=2, edgecolor='blue', linestyle='--', facecolor='none', label=f'Square Side {side:.4f}')
    ax.add_patch(square)
    
    ax.set_aspect('equal')
    if title:
        ax.set_title(title)
    else:
        ax.set_title(f"Packing for N={n} (Side: {side:.4f})")
    
    plt.legend()
    plt.savefig(f"packing_n{n}.png")
    plt.close()

def main():
    print("Loading submission...")
    tree_dict = parse_csv("submission.csv")
    
    # Visualize specific Ns
    target_ns = [10, 15, 20, 25]
    for n in target_ns:
        if n in tree_dict:
            print(f"Plotting N={n}...")
            plot_packing(tree_dict[n], n)
            
if __name__ == "__main__":
    main()
